In [1]:
import scipy
import matplotlib.pyplot as plt
import seaborn as sb
import pystan
from scipy.stats import pearsonr, zscore
import pandas as pd
import numpy as np
import os
from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl

In [44]:
directory = '/media/ak/DataOnly/TradesPurchased/'
ESDir = "".join((directory, 'ES/'))
CLDir = "".join((directory, 'CL/'))
JNIDir = "".join((directory, 'JNI/'))


In [45]:
os.listdir(directory)

['JNI',
 'ES',
 'Trades & Quotes-07-15-2020 1_56_41.zip',
 'Trades & Quotes-07-15-2020 2_33_56.zip',
 'Trades & Quotes-07-15-2020 8_45_36.zip',
 'TY',
 'CL',
 'Trades & Quotes-07-15-2020 11_41_52.zip',
 'Trades & Quotes-07-15-2020 1_56_48.zip',
 'FLG',
 'Trades & Quotes-07-15-2020 1_56_35.zip',
 'FGBL',
 'STXE',
 'Trades & Quotes-07-15-2020 11_42_08.zip',
 'Trades & Quotes-07-15-2020 2_34_40.zip']

In [37]:
file = "".join((JNIDir,'JNIM2023.csv',))

In [38]:
testDF =pd.read_csv(file)

In [39]:
testDF.columns.values

array(['Date', 'Time', 'Type', 'Price', 'Volume', 'Bid Price', 'Bid Size',
       'Ask Price', 'Ask Size'], dtype=object)

In [43]:
testDF[testDF['Type']=='Quote']

,Date,Time,Type,Price,Volume,Bid Price,Bid Size,Ask Price,Ask Size
0,05/23/2019,19:24:47.347533,Quote,NaN,NaN,NaN,NaN,20230.0,8.0
1,05/24/2019,01:07:40.927820,Quote,NaN,NaN,NaN,NaN,20240.0,1.0
2,05/24/2019,01:07:59.048161,Quote,NaN,NaN,NaN,NaN,20230.0,8.0
3,05/24/2019,05:25:06.434737,Quote,NaN,NaN,NaN,NaN,20240.0,1.0
4,05/27/2019,20:46:13.679218,Quote,NaN,NaN,NaN,NaN,20530.0,8.0
5,05/27/2019,20:46:38.598842,Quote,NaN,NaN,NaN,NaN,20540.0,1.0
6,05/27/2019,20:46:38.694834,Quote,NaN,NaN,NaN,NaN,20530.0,8.0
7,05/27/2019,20:46:45.003290,Quote,NaN,NaN,NaN,NaN,20540.0,1.0
8,05/27/2019,20:46:45.099249,Quote,NaN,NaN,NaN,NaN,20530.0,8.0
9,05/27/2019,20:48:19.171267,Quote,NaN,NaN,NaN,NaN,20540.0,1.0
